In [2]:
import pandas as pd
import os
import re

In [3]:
# CALCULATE BUDGETS FOR EACH AREA
# link to source: https://app.clarity.so/rawdao/work/6

init_budget = {}

init_budget['governance'] = 1.5
init_budget['finance'] = 1.5
init_budget['curation'] = 2.5
init_budget['internal relations'] = 1
init_budget['marketing & pr'] = 1

perf_tranche = {}

perf_tranche['governance'] =  0
perf_tranche['finance'] = 0
perf_tranche['curation'] = 0
perf_tranche['internal relations'] = 0
perf_tranche['marketing & pr'] = 0

perf_score = {}

perf_score['governance'] =  0
perf_score['finance'] = 0
perf_score['curation'] = 0
perf_score['internal relations'] = 0
perf_score['marketing & pr'] = 0


final_budget = {}
sum = 0

for area in init_budget:
  final_budget[area] = init_budget[area]*(1-perf_tranche[area]) + init_budget[area]*(perf_tranche[area]*perf_score[area])
  sum = sum + final_budget[area]

print(final_budget)
print(sum)

{'governance': 1.5, 'finance': 1.5, 'curation': 2.5, 'internal relations': 1, 'marketing & pr': 1}
7.5


In [16]:
# READ AND CLEAN COORDINAPE DATA
dfs = {}
voting_members = {}
total_give = {}

for file in os.listdir('./coordinape_data'):
  try:
    # read data from coordinape files  
    key = re.search('DAO-(.+?)-', file).group(1).lower()
    # print(key)
    dfs[key] = pd.read_csv('./coordinape_data/'+ file)
    
    # calculate voting stats for each area
    voting_members[key] = int(dfs[key]['sent'].astype(bool).sum(axis=0))
    total_give[key] = int(dfs[key]['received'].sum())

    # calculate compensation per area
    dfs[key] = dfs[key].sort_values(by=['received'], ascending = False) # sort
    dfs[key] = dfs[key][['name','address','received']] # select relevant cols
    dfs[key]['percent_comp'] = dfs[key]['received']/total_give[key] # calculate percentage comp.
    dfs[key]['eth_compensation'] = dfs[key]['percent_comp'] * final_budget[key] # calculate actual ETH comp.
    dfs[key] = dfs[key][dfs[key]['eth_compensation'] > 0] # filter people with 0 compensation
    dfs[key]['area'] = key
    dfs[key]['total_budget_of_area'] = final_budget[key]
  except NameError:
    print(file + ' -> file skipped')
    print(key)
    print(NameError)

# prepare area breakdown for export
df_export = pd.concat(dfs)

# merge everything and calculate compensation per member
comp_df = pd.concat(dfs)
comp_df = comp_df[['name','address','eth_compensation']]
comp_df = comp_df.groupby(by=['name','address']).sum().reset_index()
comp_df = comp_df.sort_values(by=['eth_compensation'], ascending = False)

print('# of members who voted per area: ' + str(voting_members))
print('total votes per area: ' + str(total_give))

# of members who voted per area: {'curation': 18, 'finance': 15, 'governance': 15, 'internal relations': 13, 'marketing & pr': 21}
total votes per area: {'curation': 1676, 'finance': 1328, 'governance': 1256, 'internal relations': 1152, 'marketing & pr': 1906}


In [22]:
# print votes per member per area
df_export[['name','total_budget_of_area','received','percent_comp','eth_compensation']]

name  total_budget_of_area  received  \
curation       24           hddamico#7928                   2.5       179   
               13   alexcartagenamex#4215                   2.5       173   
               11        Zak_Krevitt#7060                   2.5       145   
               25                lucaspon                   2.5       137   
               2             Freddie#0100                   2.5       130   
...                                   ...                   ...       ...   
marketing & pr 28        samedixsnft#8110                   1.0        16   
               31          thebonnie#3404                   1.0         7   
               22           foto.eth#8472                   1.0         7   
               5   Let’s Not Know Us#1036                   1.0         5   
               0               Coordinape                   1.0         2   

                   percent_comp  eth_compensation  
curation       24      0.106802          0.267005  
               13      0.103222          0.258055  
               11      0.086516          0.216289  
               25      0.081742          0.204356  
               2       0.077566          0.193914  
...                         ...               ...  
marketing & pr 28      0.008395          0.008395  
               31      0.003673          0.003673  
               22      0.003673          0.003673  
               5       0.002623          0.002623  
               0       0.001049          0.001049  

[130 rows x 5 columns]

In [21]:
# print compensation per member
comp_df[['name','eth_compensation']]

,name,eth_compensation
25,lucaspon,1.295928
4,Lapsus#4632,0.902047
24,hddamico#7928,0.591126
20,claudia#0273,0.478836
13,alexcartagenamex#4215,0.417482
29,shanelavalette#2830,0.316618
23,glory#5406,0.310769
18,chadwicktyler#3331,0.287992
27,mmiliszkiewicz#5782,0.252136
2,Freddie#0100,0.239035


In [17]:
# EXPORT RESULTS TO EXCEL
from pathlib import Path
Path("./outputs").mkdir(parents=True, exist_ok=True)

df_export.to_excel("./outputs/compensation_per_area.xlsx")
comp_df.to_excel("./outputs/compensation_by_member.xlsx")